In [44]:
###################the graph of computaiton###################
# input -> conv2d -> BatchNorm2d -> relu -> conv2d -> BatchNorm2d -> relu
#       -> view -> linear -> relu -> linear 
#       -> MSELoss
#       -> loss

import torch 
import visdom
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# prepare the visdom envirement
viz = visdom.Visdom(env='beamforming', server='10.15.89.41', port=38723, use_incoming_socket=False)

# parameter setting 
G = 3
N = 50
K = 5
batchSize = 200
testFlag = True  # if True: load the pre-trained model
PATH = '/BeamformingNet.pt'

class BeamformingDataset(Dataset):
# 1. Prepare Dataset
    def __init__(self, filepath):
        xy = np.loadtxt(filepath, delimiter = ',', dtype = np.float32, skiprows=1)
        self.len = xy.shape[0]
        self.x_data = torch.from_numpy(xy[:, :-2*K*G])
        self.y_data = torch.from_numpy(xy[:, 2*N*K*G:])
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

## Load the trainingSet and testingSet
trainingSet = BeamformingDataset('../N=50/trainSet.csv')
testingSet = BeamformingDataset('../N=50/testSet.csv')
train_loader = DataLoader(dataset = trainingSet, batch_size=batchSize, shuffle=True, num_workers=56)
test_loader = DataLoader(dataset = testingSet, batch_size=batchSize, shuffle=False, num_workers=56)

# 2. Design Model
class BeamformingNet(torch.nn.Module):
    def __init__(self):
        super(BeamformingNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 12, kernel_size = 3, padding = 1)
        # self.conv1 = torch.nn.DataParallel(self.conv1)
        self.conv2 = torch.nn.Conv2d(12, 12, kernel_size = 3, padding = 1)
        # self.conv2 = torch.nn.DataParallel(self.conv2)
        self.conv3 = torch.nn.Conv2d(12, 8, kernel_size = 3, padding = 1)
      
        self.norm1 = torch.nn.BatchNorm2d(12, eps = 1e-03, momentum = 0.99)
        self.norm2 = torch.nn.BatchNorm2d(8, eps = 1e-03, momentum = 0.99)
        self.fc = torch.nn.Linear(12000, 2*G*K)
        
    def forward(self, x):
        BATCH_SIZE = x.size(0)
        
        x = x.view(BATCH_SIZE, 1, 1, -1)
        x = F.relu(self.norm1(self.conv1(x)))
        x = F.relu(self.norm1(self.conv2(x)))
        x = F.relu(self.norm2(self.conv3(x)))
        x = x.view(BATCH_SIZE, -1) #flatten
        x = self.fc(x)
        # x = F.sigmoid(x)
        return x
    
model = BeamformingNet()
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

# 3. Construct Loss and Optimizer
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 1e-3)

# 4. Train and Test
def train(epoch):
    running_loss = 0.0
    train_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        inputs, target = inputs.to(device), target.to(device)
        optimizer.zero_grad()
        
        # forward + backward + update
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        train_loss += loss.item()
        if batch_idx % 2 == 1:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / 2))
            running_loss = 0.0
    viz.line([train_loss], [epoch], win='train_loss', update='append')
            
def test():
    with torch.no_grad():
        running_loss = 0.0
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            
            # compute the loss
            outputs = model(inputs)
            loss = criterion(outputs, target)
            running_loss += loss.item()      
        print('Running loss on test set: %.3f' % running_loss)
        return running_loss
    
def testSaveData():
    outputData = []
    with torch.no_grad():
        running_loss = 0.0
        for data in test_loader:
            inputs, target = data
            inputs, target = inputs.to(device), target.to(device)
            
            # compute the loss
            outputs = model(inputs)
            outputData.append(outputs.numpy())
            loss = criterion(outputs, target)
            running_loss += loss.item()      
        print('Running loss on test set: %.3f' % running_loss)
        outputData = np.reshape(outputData, (-1, 30))
        print(np.shape(outputData))
        np.savetxt('outputData.csv', outputData)
        
def main():
    viz.line([0], [0], win='train_loss', opts=dict(title='train_loss'))
    # if testFlay: True load the pre-trained model and test 
    if testFlag:
        checkpoint = torch.load(PATH)
        model.load_state_dict(checkpoint['model'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epochs = checkpoint['epoch']
        testSaveData()
        return
        
    minLoss = 10000
    for epoch in range(20):
        train(epoch)
        testLoss = test()
        # save the best parameters of the trained model
        if testLoss < minLoss:
            minLoss = testLoss
            print(minLoss)
            state = {'model':model.state_dict(), 'optimizer':optimizer.state_dict(), 'epoch':epoch}
            torch.save(state, PATH)
    
if __name__ == '__main__':
    main()

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


Running loss on test set: 0.780
(5000, 30)


In [6]:
# Generate the dataset
import numpy as np

np.random.seed(0)

G = 3
N = 50
K = 5
trainingSample = 20000
testingSample = 5000

trainingInputs = np.random.randn(trainingSample, 2*N*G*K)
testingInputs = np.random.randn(testingSample, 2*N*G*K)

# outputs size: 2*G*k
a = np.random.randn(2*N*G*K, 2*G*K)
trainingOutputs = np.dot(trainingInputs, a)
testingOutputs = np.dot(testingInputs, a)

trainingData = np.hstack((trainingInputs, trainingOutputs))
testingData = np.hstack((testingInputs, testingOutputs))

np.savetxt('trainingData.csv.gz', trainingData, fmt='%5f', delimiter=',')
np.savetxt('testingData.csv.gz', testingData, fmt='%5f', delimiter=',')

In [8]:
import torch 
import visdom
import numpy as np

vis = visdom.Visdom(env='test', server='10.15.89.41', port=38720, use_incoming_socket=False)

##
trace = dict(x=[1, 2, 3], y=[4, 5, 6], mode="markers+lines", type='custom',
             marker={'color': 'red', 'symbol': 104, 'size': "10"},
             text=["one", "two", "three"], name='1st Trace')
layout = dict(title="First Plot", xaxis={'title': 'x'}, yaxis={'title': 'y'})

vis._send({'data': [trace], 'layout': layout, 'win': 'mywin'})

Setting up a new session...
Without the incoming socket you cannot receive events from the server or register event handlers to your Visdom client.


'mywin'

In [43]:
import numpy as np

a=np.reshape(np.random.randn(18),(3,3,2))
print(a.shape())
b = np.reshape(a, (-1, 2))
print(b)

TypeError: 'tuple' object is not callable

In [ ]:
ggg